1. Создайте список из 1000 случайных строк, состоящих из латинских букв.

In [44]:
import random
import string

In [45]:
def generate_random_strs(min_len, max_len, count=1000, char_set=string.ascii_letters):
    if min_len >= max_len:
        raise ValueError("bad min/max values")
    
    return [''.join(random.choice(char_set) for i in range(min_len, random.randint(min_len + 1, max_len))) \
            for i in range(count)]

In [47]:
rand_strs = generate_random_strs(1, 100, 1000)
print(rand_strs)

['TZHpph', 'nOzpPjJpuRgFXzbrZuaizJfqUOQjQi', 'XaxZMLjnhfSIrQDZJPTMuUbHHjazNnQKggaSLaWkNvxuMLLbxfUmSgU', 'EqYhsLWMq', 'ooDXWXNRGcpqzWQwHREqZrOyVpVfXapgYE', 'ToIWfNeVAPmORarWWEBdpxAGhMdfQGhZnHrCeKHaMkeFtWJyiKYuIrhxZBOIPSPuRGAnedjfQZUyAf', 'EecgehJYkRr', 'OAXrgqNNexPbCUaPQiltcMNlxSCyjjPyxHvFfoMMdFFuYxXigkhO', 'sKcYMFwSAthkjChMEfoAQUwHprKvVdKovjHqRglkUUgARGsPTuODwSKKHPRtdTYIpNZ', 'NfEDTWxZEIqpH', 'naNLiKsCMfRrMqIfzAcwRnICAQrYoRLhkVeBFmHgBOgzRihaWsaeDqkfKMTZZcHLvdEGkatsjbfoxHZC', 'yzfweHRGVKLuCtZ', 'YqFmu', 'uoSKtuQnJggFfGetXXEytiaFwgKtBKomvHQITanwzzkMFnAeVYMujiZzVNcfcddfBSvfFbuejMuRiupkydB', 'YGlC', 'VLKTIpEPhFTVK', 'swhLorFfoBpeWYuULcWSRrNACoJvOQZqAvBOWYGbNxmoTxWiRYrQkCSUbQabjYnqmDl', 'LQirCikkWYQeQvOepralhvLCaGHIbpxSanzodbrzgspugTcwqvGViiHNtGSpaRgCVmVXLVuvbLYUYRpJivq', 'obWXfGqTFESJlURnZPPPBiQlLdaoHpKhqagHLqbIPQzPHFxhiGmvoqdjV', 'IIjcrYRfgAgrVaBYyHjmCWwktZfXDVToVUqegFSQtgujWiKhEvEcPMdfLnlz', 'ivr', 'kAzERmEjuaWIOiBUNaJehAmfxthdYxYqsQRxRlYauopyqgHHboWMsdRZlPlXzvgAzOQNBpZXihgTkxzCHUGFzexvp